1. Import required libraries and the dataset

In [523]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [524]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import backend
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.optimizers import adam,sgd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [525]:
statastics = []

2. Visualise a sample of dataset

In [526]:
dataset = pd.read_csv('titanic.csv')
dataset

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,?,C,?,328,?
1305,3,0,"Zabour, Miss. Thamine",female,?,1,0,2665,14.4542,?,C,?,?,?
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.225,?,C,?,304,?
1307,3,0,"Zakarian, Mr. Ortin",male,27,0,0,2670,7.225,?,C,?,?,?


In [527]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1309 non-null object
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1309 non-null object
cabin        1309 non-null object
embarked     1309 non-null object
boat         1309 non-null object
body         1309 non-null object
home.dest    1309 non-null object
dtypes: int64(4), object(10)
memory usage: 143.3+ KB


In [528]:
dataset.describe()

,pclass,survived,sibsp,parch
count,1309.000000,1309.000000,1309.000000,1309.000000
mean,2.294882,0.381971,0.498854,0.385027
std,0.837836,0.486055,1.041658,0.865560
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000
50%,3.000000,0.000000,0.000000,0.000000
75%,3.000000,1.000000,1.000000,0.000000
max,3.000000,1.000000,8.000000,9.000000


3. Data preprocessing

In [529]:
# As below columns are not contributing to the prediction of survival, dropping them from the data set
# name ,ticket, fare, cabin, boat, body, home.dest

dataset = dataset.drop(['name','ticket','fare','cabin','boat','body','home.dest'],axis =1)

dataset

,pclass,survived,sex,age,sibsp,parch,embarked
0,1,1,female,29,0,0,S
1,1,1,male,0.9167,1,2,S
2,1,0,female,2,1,2,S
3,1,0,male,30,1,2,S
4,1,0,female,25,1,2,S
...,...,...,...,...,...,...,...
1304,3,0,female,14.5,1,0,C
1305,3,0,female,?,1,0,C
1306,3,0,male,26.5,0,0,C
1307,3,0,male,27,0,0,C


In [530]:
dataset.describe()
dataset.describe(include=['object'])

,sex,age,embarked
count,1309,1309,1309
unique,2,99,4
top,male,?,S
freq,843,263,914


In [531]:
dataset.isin(["?"]).sum()

pclass        0
survived      0
sex           0
age         263
sibsp         0
parch         0
embarked      2
dtype: int64

In [532]:
# since we have some data cells which still have "?" mark symbol, this needs to be filled with appropriate value
# dataset[‘embarked’].replace(0, np.nan, inplace= True)
common_value="S"
median = dataset.loc[dataset['age']!="?", 'age'].median()
# print(median)
dataset['age'].replace("?", median, inplace=True)
dataset['embarked'].replace("?", "S", inplace=True)


In [533]:
dataset.isin(["?"]).sum() #Successfully replaced the both values and can be seen the same below

pclass      0
survived    0
sex         0
age         0
sibsp       0
parch       0
embarked    0
dtype: int64

In [534]:
#Still we see sex column's data is categorical, it should be convereted using lable encoder
label_encoder_sex = LabelEncoder()
dataset.iloc[:,2]  = label_encoder_sex.fit_transform(dataset.iloc[:,2])

In [535]:
embarked_dict={"S":1,"C":2,"Q":3}
dataset['embarked']=[embarked_dict[item] for item in dataset['embarked']] 

In [536]:
dataset

,pclass,survived,sex,age,sibsp,parch,embarked
0,1,1,0,29,0,0,1
1,1,1,1,0.9167,1,2,1
2,1,0,0,2,1,2,1
3,1,0,1,30,1,2,1
4,1,0,0,25,1,2,1
...,...,...,...,...,...,...,...
1304,3,0,0,14.5,1,0,2
1305,3,0,0,28,1,0,2
1306,3,0,1,26.5,0,0,2
1307,3,0,1,27,0,0,2


In [537]:
# As preprocessing of the data is completed, it is time to split the data into test and train
import sklearn.model_selection as cross_validate
from sklearn.preprocessing import StandardScaler
X=dataset.drop('survived', axis=1).values.astype(float)
y=dataset['survived'].values.astype(int)
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
X_train, X_test, y_train, y_test = cross_validate.train_test_split(X, y, test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [538]:
X_test.shape

(262, 6)

# Combination 1
1. Network Depth : 1 Hidden Layer of 64 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Relu for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : Adam with lr 1e-03

In [539]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.Adam(lr=1e-03)
batch_size = 64
num_folds = 3

In [540]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(64,input_dim = 6, activation = 'relu'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 2.0396 - acc: 0.5234
Epoch 2/20
1047/1047 [==============================] - 0s 64us/sample - loss: 1.9628 - acc: 0.6523
Epoch 3/20
1047/1047 [==============================] - 0s 70us/sample - loss: 2.0620 - acc: 0.7268
Epoch 4/20
1047/1047 [==============================] - 0s 67us/sample - loss: 1.7766 - acc: 0.7507
Epoch 5/20
1047/1047 [==============================] - 0s 73us/sample - loss: 1.8172 - acc: 0.7603
Epoch 6/20
1047/1047 [==============================] - 0s 71us/sample - loss: 2.1406 - acc: 0.7345
Epoch 7/20
1047/1047 [==============================] - 0s 66us/sample - loss: 1.7284 - acc: 0.7584
Epoch 8/20
1047/1047 [==============================] - 0s 65us/sample - loss: 1.9381 - acc: 0.7412
Epoch 9/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.7837 - acc: 0.7641
Epoch 10/20
1047/1047 [==============================] - 0s 80us/sample - loss: 1.9539 - acc: 0.7593


In [541]:
loss_comb1, accuracy_comb1 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb1)
print("Test Accuracy:", accuracy_comb1)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4697 - acc: 0.7824
Test Loss: 0.4696569856796556
Test Accuracy: 0.78244275


In [542]:
statastics.append([loss_comb1,accuracy_comb1*100])

In [543]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.12076133]
 [0.0963901 ]
 [0.12132353]
 [0.50236195]
 [0.11783794]
 [0.502714  ]
 [0.43657216]
 [0.62478465]
 [0.09779936]
 [0.0963901 ]
 [0.27521336]
 [0.08594212]
 [0.668352  ]
 [0.25387686]
 [0.0963901 ]
 [0.12026811]
 [0.6314531 ]
 [0.10597643]
 [0.38712567]
 [0.12449419]
 [0.37273675]
 [0.21470815]
 [0.21393609]
 [0.18714786]
 [0.10617691]
 [0.21470815]
 [0.17415124]
 [0.12839168]
 [0.10931113]
 [0.26276892]
 [0.75013006]
 [0.51921666]
 [0.6104119 ]
 [0.1986131 ]
 [0.13253534]
 [0.16306141]
 [0.20279855]
 [0.24642208]
 [0.14860132]
 [0.11651945]
 [0.3876167 ]
 [0.5844358 ]
 [0.6668753 ]
 [0.38635612]
 [0.05814549]
 [0.04432216]
 [0.48129398]
 [0.14860132]
 [0.6169703 ]
 [0.40966296]
 [0.08035982]
 [0.6275956 ]
 [0.09036943]
 [0.07797378]
 [0.7512261 ]
 [0.7413407 ]
 [0.0963901 ]
 [0.747802  ]
 [0.13397273]
 [0.38666716]
 [0.60903656]
 [0.638235  ]
 [0.1141974 ]
 [0.16416967]
 [0.0963901 ]
 [0.12076133]
 [0.19012997]
 [0.11783794]
 [0.04101425]
 [0.1572757 ]
 [0.3167159 ]
 [0.14

In [544]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[160   9]
 [ 48  45]]




# Combination 2
1. Network Depth : 1 Hidden Layer of 64 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : Adam with lr 1e-03

In [545]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.Adam(lr=1e-03)
batch_size = 64
num_folds = 3

In [546]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(64,input_dim = 6, activation = 'sigmoid'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 2.0363 - acc: 0.6275
Epoch 2/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.8557 - acc: 0.6132
Epoch 3/20
1047/1047 [==============================] - 0s 73us/sample - loss: 2.0079 - acc: 0.6495
Epoch 4/20
1047/1047 [==============================] - 0s 71us/sample - loss: 2.0599 - acc: 0.7030
Epoch 5/20
1047/1047 [==============================] - 0s 70us/sample - loss: 2.1841 - acc: 0.7030
Epoch 6/20
1047/1047 [==============================] - 0s 84us/sample - loss: 1.9191 - acc: 0.7249
Epoch 7/20
1047/1047 [==============================] - 0s 112us/sample - loss: 1.8600 - acc: 0.7431
Epoch 8/20
1047/1047 [==============================] - 0s 110us/sample - loss: 1.9831 - acc: 0.7431
Epoch 9/20
1047/1047 [==============================] - 0s 107us/sample - loss: 1.8900 - acc: 0.7469
Epoch 10/20
1047/1047 [==============================] - 0s 103us/sample - loss: 2.1826 - acc: 0.7

In [547]:
loss_comb2, accuracy_comb2 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb2)
print("Test Accuracy:", accuracy_comb2)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4752 - acc: 0.8015
Test Loss: 0.4751596837553359
Test Accuracy: 0.8015267


In [548]:
statastics.append([loss_comb2,accuracy_comb2*100])

In [549]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.16944894]
 [0.10230586]
 [0.10319316]
 [0.4273548 ]
 [0.16236132]
 [0.5227198 ]
 [0.41744965]
 [0.5469593 ]
 [0.08582276]
 [0.10230586]
 [0.27347535]
 [0.13902044]
 [0.62720644]
 [0.29142827]
 [0.10230586]
 [0.11600497]
 [0.5907844 ]
 [0.10793844]
 [0.3505441 ]
 [0.11812291]
 [0.4993358 ]
 [0.33021402]
 [0.15723139]
 [0.29535148]
 [0.15853474]
 [0.33021402]
 [0.23028216]
 [0.1706444 ]
 [0.10989481]
 [0.27668983]
 [0.6754598 ]
 [0.548512  ]
 [0.4309979 ]
 [0.22605222]
 [0.12829924]
 [0.11485744]
 [0.22085786]
 [0.3969967 ]
 [0.19043753]
 [0.11392781]
 [0.34135383]
 [0.590014  ]
 [0.765391  ]
 [0.373761  ]
 [0.1100885 ]
 [0.06518847]
 [0.50336117]
 [0.19043753]
 [0.52151555]
 [0.3833013 ]
 [0.09207082]
 [0.5668069 ]
 [0.07984337]
 [0.09049341]
 [0.7586165 ]
 [0.7396359 ]
 [0.10230586]
 [0.6454941 ]
 [0.1224817 ]
 [0.3942925 ]
 [0.51261914]
 [0.6365888 ]
 [0.22360265]
 [0.20016003]
 [0.10230586]
 [0.16944894]
 [0.2137785 ]
 [0.16236132]
 [0.15199947]
 [0.14503852]
 [0.32756567]
 [0.25

In [550]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[161   8]
 [ 44  49]]




# Combination 3
1. Network Depth : 1 Hidden Layer of 64 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Relu for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : SGD with lr 1e-02 and momentum 0.95

In [551]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.SGD(momentum=0.95)
batch_size = 64
num_folds = 3

In [552]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(64,input_dim = 6, activation = 'relu'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 1.9890 - acc: 0.4833
Epoch 2/20
1047/1047 [==============================] - 0s 62us/sample - loss: 2.0316 - acc: 0.7096
Epoch 3/20
1047/1047 [==============================] - 0s 56us/sample - loss: 1.8357 - acc: 0.7555
Epoch 4/20
1047/1047 [==============================] - 0s 64us/sample - loss: 1.9702 - acc: 0.7440
Epoch 5/20
1047/1047 [==============================] - 0s 71us/sample - loss: 2.0782 - acc: 0.7402
Epoch 6/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.8381 - acc: 0.7670
Epoch 7/20
1047/1047 [==============================] - 0s 67us/sample - loss: 1.8889 - acc: 0.7698
Epoch 8/20
1047/1047 [==============================] - 0s 65us/sample - loss: 1.8526 - acc: 0.7507
Epoch 9/20
1047/1047 [==============================] - 0s 63us/sample - loss: 2.0027 - acc: 0.7526
Epoch 10/20
1047/1047 [==============================] - 0s 61us/sample - loss: 1.7998 - acc: 0.7670


In [553]:
loss_comb3, accuracy_comb3 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb3)
print("Test Accuracy:", accuracy_comb3)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4385 - acc: 0.8053
Test Loss: 0.4384832750749952
Test Accuracy: 0.8053435


In [554]:
statastics.append([loss_comb3,accuracy_comb3*100])

In [555]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.07773879]
 [0.0993326 ]
 [0.14119986]
 [0.4654579 ]
 [0.11160752]
 [0.504939  ]
 [0.3699224 ]
 [0.66996866]
 [0.08496785]
 [0.0993326 ]
 [0.24537992]
 [0.05062422]
 [0.8594787 ]
 [0.28998446]
 [0.0993326 ]
 [0.14345455]
 [0.6505176 ]
 [0.11921564]
 [0.28314596]
 [0.14997652]
 [0.5042379 ]
 [0.19662994]
 [0.4074012 ]
 [0.17960247]
 [0.06757435]
 [0.19662994]
 [0.15868488]
 [0.1308822 ]
 [0.12552398]
 [0.16802517]
 [0.8882956 ]
 [0.52121866]
 [0.70466226]
 [0.11124033]
 [0.12205091]
 [0.21234608]
 [0.17922726]
 [0.30922812]
 [0.10777777]
 [0.13726571]
 [0.2893415 ]
 [0.6639263 ]
 [0.9020386 ]
 [0.28608358]
 [0.03040829]
 [0.01639301]
 [0.49105814]
 [0.10777777]
 [0.6847094 ]
 [0.3539341 ]
 [0.06753775]
 [0.66039264]
 [0.06825599]
 [0.06323814]
 [0.9105239 ]
 [0.8669492 ]
 [0.0993326 ]
 [0.8984175 ]
 [0.16483775]
 [0.37039772]
 [0.65407187]
 [0.753953  ]
 [0.0955911 ]
 [0.12616822]
 [0.0993326 ]
 [0.07773879]
 [0.15903151]
 [0.11160752]
 [0.01366955]
 [0.10082752]
 [0.21538293]
 [0.13

In [556]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[160   9]
 [ 42  51]]




# Combination 4
1. Network Depth : 1 Hidden Layer of 64 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : SGD with lr 1e-02 and momentum 0.95

In [557]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.SGD(momentum=0.95)
batch_size = 64
num_folds = 3

In [558]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(64,input_dim = 6, activation = 'sigmoid'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 2.1396 - acc: 0.5654
Epoch 2/20
1047/1047 [==============================] - 0s 64us/sample - loss: 2.1298 - acc: 0.5989
Epoch 3/20
1047/1047 [==============================] - 0s 62us/sample - loss: 1.9097 - acc: 0.6476
Epoch 4/20
1047/1047 [==============================] - 0s 59us/sample - loss: 1.9900 - acc: 0.7135
Epoch 5/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.9100 - acc: 0.7287
Epoch 6/20
1047/1047 [==============================] - 0s 74us/sample - loss: 1.8525 - acc: 0.7450
Epoch 7/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.8792 - acc: 0.7393
Epoch 8/20
1047/1047 [==============================] - 0s 68us/sample - loss: 2.0389 - acc: 0.7412
Epoch 9/20
1047/1047 [==============================] - 0s 62us/sample - loss: 1.8768 - acc: 0.7526
Epoch 10/20
1047/1047 [==============================] - 0s 62us/sample - loss: 1.7985 - acc: 0.7593


In [559]:
loss_comb4, accuracy_comb4 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb4)
print("Test Accuracy:", accuracy_comb4)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4661 - acc: 0.8015
Test Loss: 0.4661302698477534
Test Accuracy: 0.8015267


In [560]:
statastics.append([loss_comb4,accuracy_comb4*100])

In [561]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.15244189]
 [0.08650738]
 [0.08722439]
 [0.4460741 ]
 [0.12566847]
 [0.53501475]
 [0.46789855]
 [0.6115375 ]
 [0.07323191]
 [0.08650738]
 [0.23821563]
 [0.12590414]
 [0.697689  ]
 [0.27756345]
 [0.08650738]
 [0.09774229]
 [0.6489323 ]
 [0.09110233]
 [0.40393257]
 [0.09949732]
 [0.5181236 ]
 [0.3120404 ]
 [0.12772968]
 [0.27949882]
 [0.14291272]
 [0.3120404 ]
 [0.19764185]
 [0.13175362]
 [0.09270632]
 [0.3100378 ]
 [0.73812705]
 [0.5599153 ]
 [0.50220466]
 [0.24079481]
 [0.10335258]
 [0.09176126]
 [0.19755036]
 [0.36266792]
 [0.17080867]
 [0.09602579]
 [0.3947597 ]
 [0.58768696]
 [0.7728973 ]
 [0.4267456 ]
 [0.10075033]
 [0.05708712]
 [0.51614875]
 [0.17080867]
 [0.58860755]
 [0.43551037]
 [0.07824489]
 [0.628796  ]
 [0.06852099]
 [0.07698148]
 [0.79560626]
 [0.78923583]
 [0.08650738]
 [0.7134497 ]
 [0.10312447]
 [0.40530592]
 [0.580474  ]
 [0.6776749 ]
 [0.21121317]
 [0.17933834]
 [0.08650738]
 [0.15244189]
 [0.19131243]
 [0.12566847]
 [0.15761441]
 [0.1322552 ]
 [0.32583702]
 [0.23

In [562]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[160   9]
 [ 43  50]]




# Combination 5
1. Network Depth : 2 Hidden Layer of 32 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Relu for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : Adam with lr 1e-03

In [563]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.Adam(lr=1e-03)
batch_size = 64
num_folds = 3

In [564]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(32,input_dim = 6, activation = 'relu'),
tf.keras.layers.Dense(32,activation = 'relu'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 2.3485 - acc: 0.4451
Epoch 2/20
1047/1047 [==============================] - 0s 66us/sample - loss: 2.1031 - acc: 0.5521
Epoch 3/20
1047/1047 [==============================] - 0s 60us/sample - loss: 2.2592 - acc: 0.6667
Epoch 4/20
1047/1047 [==============================] - 0s 72us/sample - loss: 2.1037 - acc: 0.7163
Epoch 5/20
1047/1047 [==============================] - 0s 74us/sample - loss: 2.0694 - acc: 0.7249
Epoch 6/20
1047/1047 [==============================] - 0s 80us/sample - loss: 2.1209 - acc: 0.7287
Epoch 7/20
1047/1047 [==============================] - 0s 81us/sample - loss: 1.9810 - acc: 0.7412
Epoch 8/20
1047/1047 [==============================] - 0s 110us/sample - loss: 2.0618 - acc: 0.7383
Epoch 9/20
1047/1047 [==============================] - 0s 70us/sample - loss: 1.9427 - acc: 0.7469
Epoch 10/20
1047/1047 [==============================] - 0s 73us/sample - loss: 1.7595 - acc: 0.7564

In [565]:
loss_comb5, accuracy_comb5 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb5)
print("Test Accuracy:", accuracy_comb5)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4564 - acc: 0.8053
Test Loss: 0.45638758717602446
Test Accuracy: 0.8053435


In [566]:
statastics.append([loss_comb5,accuracy_comb5*100])

In [567]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.12018222]
 [0.09834394]
 [0.13146523]
 [0.49875316]
 [0.09998265]
 [0.42287734]
 [0.40817982]
 [0.6677789 ]
 [0.09913227]
 [0.09834394]
 [0.25771356]
 [0.08454958]
 [0.7859278 ]
 [0.24948892]
 [0.09834394]
 [0.11674675]
 [0.63976467]
 [0.10699427]
 [0.40615022]
 [0.12034532]
 [0.4407191 ]
 [0.24428147]
 [0.34502923]
 [0.19332981]
 [0.11021677]
 [0.24428147]
 [0.1942369 ]
 [0.11238956]
 [0.10938141]
 [0.30266988]
 [0.7431226 ]
 [0.46202067]
 [0.6325366 ]
 [0.22508022]
 [0.10942164]
 [0.26064175]
 [0.19871491]
 [0.31711861]
 [0.14103553]
 [0.11409166]
 [0.39931345]
 [0.5820149 ]
 [0.68450886]
 [0.4147233 ]
 [0.04686967]
 [0.03431699]
 [0.39795882]
 [0.14103553]
 [0.6729883 ]
 [0.36050653]
 [0.0786843 ]
 [0.6539321 ]
 [0.08957171]
 [0.07547182]
 [0.82304347]
 [0.7732045 ]
 [0.09834394]
 [0.7625966 ]
 [0.12810153]
 [0.43292552]
 [0.6193636 ]
 [0.57744503]
 [0.10023052]
 [0.15363142]
 [0.09834394]
 [0.12018222]
 [0.18333489]
 [0.09998265]
 [0.03785455]
 [0.12698585]
 [0.41046178]
 [0.13

In [568]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[162   7]
 [ 44  49]]




# Combination 6
1. Network Depth : 2 Hidden Layer of 32 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : Adam with lr 1e-03

In [569]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.Adam(lr=1e-03)
batch_size = 64
num_folds = 3

In [570]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(32,input_dim = 6, activation = 'sigmoid'),
tf.keras.layers.Dense(32,activation = 'sigmoid'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 1.9516 - acc: 0.5530
Epoch 2/20
1047/1047 [==============================] - 0s 62us/sample - loss: 2.0081 - acc: 0.6113
Epoch 3/20
1047/1047 [==============================] - 0s 65us/sample - loss: 1.9401 - acc: 0.6113
Epoch 4/20
1047/1047 [==============================] - 0s 59us/sample - loss: 2.1149 - acc: 0.6113
Epoch 5/20
1047/1047 [==============================] - 0s 71us/sample - loss: 2.1946 - acc: 0.6113
Epoch 6/20
1047/1047 [==============================] - 0s 78us/sample - loss: 2.1078 - acc: 0.6122
Epoch 7/20
1047/1047 [==============================] - 0s 79us/sample - loss: 2.1381 - acc: 0.6227
Epoch 8/20
1047/1047 [==============================] - 0s 79us/sample - loss: 1.7045 - acc: 0.6724
Epoch 9/20
1047/1047 [==============================] - 0s 79us/sample - loss: 2.1662 - acc: 0.7068
Epoch 10/20
1047/1047 [==============================] - 0s 75us/sample - loss: 1.9552 - acc: 0.6944


In [571]:
loss_comb6, accuracy_comb6 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb6)
print("Test Accuracy:", accuracy_comb6)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4780 - acc: 0.7786
Test Loss: 0.4780400067795324
Test Accuracy: 0.77862597


In [572]:
statastics.append([loss_comb6,accuracy_comb6*100])

In [573]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.17728746]
 [0.10751939]
 [0.10912177]
 [0.439236  ]
 [0.1548318 ]
 [0.48223326]
 [0.4047818 ]
 [0.5497317 ]
 [0.09621146]
 [0.10751939]
 [0.32538438]
 [0.15050948]
 [0.63379616]
 [0.3198036 ]
 [0.10751939]
 [0.11798632]
 [0.5831101 ]
 [0.11176851]
 [0.34367406]
 [0.11964303]
 [0.5009772 ]
 [0.35728207]
 [0.17024702]
 [0.32378083]
 [0.16753724]
 [0.35728207]
 [0.22870612]
 [0.1609053 ]
 [0.11326268]
 [0.29934263]
 [0.66962075]
 [0.5049704 ]
 [0.4510135 ]
 [0.27163708]
 [0.12727022]
 [0.13074204]
 [0.22439924]
 [0.41678095]
 [0.19635707]
 [0.11637107]
 [0.33495244]
 [0.5748912 ]
 [0.7539034 ]
 [0.36563897]
 [0.12660453]
 [0.08192265]
 [0.46496215]
 [0.19635707]
 [0.5290145 ]
 [0.37355363]
 [0.10001418]
 [0.5652145 ]
 [0.09201133]
 [0.09888414]
 [0.728204  ]
 [0.7054772 ]
 [0.10751939]
 [0.6497627 ]
 [0.12308204]
 [0.4505534 ]
 [0.5235072 ]
 [0.62220985]
 [0.25155777]
 [0.20528781]
 [0.10751939]
 [0.17728746]
 [0.21785408]
 [0.1548318 ]
 [0.1783221 ]
 [0.15599832]
 [0.41368356]
 [0.28

In [574]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[158  11]
 [ 47  46]]




# Combination 7
1. Network Depth : 2 Hidden Layer of 32 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Relu for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : SGD with lr 1e-02 and momentum 0.95

In [575]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.SGD(momentum=0.95)
batch_size = 64
num_folds = 3

In [576]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(32,input_dim = 6, activation = 'relu'),
tf.keras.layers.Dense(32,activation = 'relu'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 1.9754 - acc: 0.5167
Epoch 2/20
1047/1047 [==============================] - 0s 63us/sample - loss: 2.3372 - acc: 0.6982
Epoch 3/20
1047/1047 [==============================] - 0s 61us/sample - loss: 2.0129 - acc: 0.7326
Epoch 4/20
1047/1047 [==============================] - 0s 63us/sample - loss: 2.1419 - acc: 0.7431
Epoch 5/20
1047/1047 [==============================] - 0s 66us/sample - loss: 2.0675 - acc: 0.7517
Epoch 6/20
1047/1047 [==============================] - 0s 71us/sample - loss: 1.9339 - acc: 0.7106
Epoch 7/20
1047/1047 [==============================] - 0s 73us/sample - loss: 2.0028 - acc: 0.7230
Epoch 8/20
1047/1047 [==============================] - 0s 69us/sample - loss: 1.7715 - acc: 0.7650
Epoch 9/20
1047/1047 [==============================] - 0s 70us/sample - loss: 2.2548 - acc: 0.7354
Epoch 10/20
1047/1047 [==============================] - 0s 69us/sample - loss: 2.0339 - acc: 0.7545


In [577]:
loss_comb7, accuracy_comb7 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb7)
print("Test Accuracy:", accuracy_comb7)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4543 - acc: 0.7901
Test Loss: 0.4543302027323774
Test Accuracy: 0.7900763


In [578]:
statastics.append([loss_comb7,accuracy_comb7*100])

In [579]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[7.91613460e-02]
 [1.11454606e-01]
 [1.59166843e-01]
 [4.39565361e-01]
 [1.20076060e-01]
 [4.27458137e-01]
 [4.10153091e-01]
 [6.83963180e-01]
 [8.66192281e-02]
 [1.11454606e-01]
 [2.30599523e-01]
 [4.82716262e-02]
 [8.40990782e-01]
 [3.00149083e-01]
 [1.11454606e-01]
 [1.43882722e-01]
 [6.53586030e-01]
 [1.26982242e-01]
 [3.38167131e-01]
 [1.48273677e-01]
 [4.25412655e-01]
 [1.84557229e-01]
 [4.18527156e-01]
 [1.56731129e-01]
 [6.66213036e-02]
 [1.84557229e-01]
 [1.47287995e-01]
 [1.40176773e-01]
 [1.31054372e-01]
 [1.87619984e-01]
 [8.17532301e-01]
 [4.38173950e-01]
 [7.10619211e-01]
 [9.58828628e-02]
 [1.23939008e-01]
 [2.72523344e-01]
 [1.58937186e-01]
 [2.72567749e-01]
 [1.08746082e-01]
 [1.39531255e-01]
 [3.36238384e-01]
 [4.75528002e-01]
 [7.48182416e-01]
 [3.35167885e-01]
 [2.14204490e-02]
 [1.49667859e-02]
 [4.22701329e-01]
 [1.08746082e-01]
 [7.00744987e-01]
 [3.95857155e-01]
 [7.86951184e-02]
 [6.68426335e-01]
 [6.64536953e-02]
 [7.37769604e-02]
 [8.67711186e-01]
 [8.090890

In [580]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[163   6]
 [ 49  44]]




# Combination 8
1. Network Depth : 2 Hidden Layer of 32 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : SGD with lr 1e-02 and momentum 0.95

In [581]:
# Model configuration and hyperparameters
no_epochs = 20
optimizer=tf.keras.optimizers.SGD(momentum=0.95)
batch_size = 64
num_folds = 3

In [582]:
#choosing sequential model
model=tf.keras.Sequential([
tf.keras.layers.Dense(32,input_dim = 6, activation = 'sigmoid'),
tf.keras.layers.Dense(32,activation = 'sigmoid'),
tf.keras.layers.Dense(1,activation = 'sigmoid'),
tf.keras.layers.Dropout(0.25)])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=batch_size, shuffle=True)



Epoch 1/20
1047/1047 [==============================] - 1s 1ms/sample - loss: 2.0226 - acc: 0.6113
Epoch 2/20
1047/1047 [==============================] - 0s 63us/sample - loss: 2.0535 - acc: 0.6113
Epoch 3/20
1047/1047 [==============================] - 0s 68us/sample - loss: 2.2280 - acc: 0.6113
Epoch 4/20
1047/1047 [==============================] - 0s 73us/sample - loss: 2.0482 - acc: 0.6113
Epoch 5/20
1047/1047 [==============================] - 0s 74us/sample - loss: 2.0459 - acc: 0.6113
Epoch 6/20
1047/1047 [==============================] - 0s 74us/sample - loss: 1.9565 - acc: 0.6380
Epoch 7/20
1047/1047 [==============================] - 0s 76us/sample - loss: 2.0306 - acc: 0.6199
Epoch 8/20
1047/1047 [==============================] - 0s 74us/sample - loss: 1.9085 - acc: 0.6208
Epoch 9/20
1047/1047 [==============================] - 0s 88us/sample - loss: 2.0816 - acc: 0.6590
Epoch 10/20
1047/1047 [==============================] - 0s 114us/sample - loss: 1.9675 - acc: 0.6925

In [583]:
loss_comb8, accuracy_comb8 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb8)
print("Test Accuracy:", accuracy_comb8)

262/262 [==============================] - 1s 3ms/sample - loss: 0.4787 - acc: 0.7939
Test Loss: 0.4786671927867045
Test Accuracy: 0.79389316


In [584]:
statastics.append([loss_comb8,accuracy_comb8*100])

In [585]:
pred_test = model.predict(X_test)
print(pred_test)

# #pred_arr is the predicted values for the input_test
# #The argmax will give the predicted value for the corresping input data

pred_test = (pred_test > 0.5)   


[[0.17762262]
 [0.11373773]
 [0.11933479]
 [0.5142949 ]
 [0.15671256]
 [0.53729147]
 [0.46338156]
 [0.5894226 ]
 [0.10869631]
 [0.11373773]
 [0.32066056]
 [0.16338587]
 [0.6694    ]
 [0.30031684]
 [0.11373773]
 [0.12046063]
 [0.6050463 ]
 [0.11641705]
 [0.43796393]
 [0.1215578 ]
 [0.53911114]
 [0.3091343 ]
 [0.16315353]
 [0.29031205]
 [0.1723446 ]
 [0.3091343 ]
 [0.21708438]
 [0.1600214 ]
 [0.11737677]
 [0.4291953 ]
 [0.69241065]
 [0.5510672 ]
 [0.5447106 ]
 [0.4169021 ]
 [0.1313765 ]
 [0.1352224 ]
 [0.20404002]
 [0.358095  ]
 [0.1881586 ]
 [0.11939827]
 [0.4332599 ]
 [0.575893  ]
 [0.74519396]
 [0.44900396]
 [0.15184751]
 [0.10123813]
 [0.5261802 ]
 [0.1881586 ]
 [0.57821584]
 [0.44666237]
 [0.10924405]
 [0.5970851 ]
 [0.10539576]
 [0.10860226]
 [0.72941905]
 [0.70822585]
 [0.11373773]
 [0.68503237]
 [0.12385738]
 [0.5073144 ]
 [0.58077043]
 [0.6368289 ]
 [0.24975601]
 [0.19317096]
 [0.11373773]
 [0.17762262]
 [0.20029664]
 [0.15671256]
 [0.33151567]
 [0.17044103]
 [0.48106942]
 [0.26

In [586]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[153  16]
 [ 38  55]]




# Combination 9
CROSS VALIDATION
1. Network Depth : 2 Hidden Layer of 32 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : ADAM with Binarycrossentropy loss

In [587]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(32, init = "uniform", activation = 'relu',input_dim = 6))
    classifier.add(Dropout(p = 0.25))
    classifier.add(Dense(32, init = "uniform", activation = 'relu'))
    classifier.add(Dropout(p = 0.25))
    classifier.add(Dense(output_dim = 1, init = "uniform", activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [588]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 64, epochs = 20)

In [589]:
accuracies_64 = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [590]:
mean_32 = accuracies_64.mean()
variance_32 = accuracies_64.std()
mean_32

0.7937454342842102

In [591]:
variance_32

0.039804605434327636

In [592]:
loss_comb9, accuracy_comb9 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb9)
print("Test Accuracy:", accuracy_comb9)

262/262 [==============================] - 0s 119us/sample - loss: 0.4787 - acc: 0.7939
Test Loss: 0.4786671927867045
Test Accuracy: 0.79389316


In [593]:
statastics.append([loss_comb9,accuracy_comb9*100])

In [594]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[153  16]
 [ 38  55]]




Combination 9
Network Depth : 1 Hidden Layer of 64 units
Regularisations : Dropout of ratio 0.25
Activations : Sigmoid for every hidden layer (singmoid for final layer)
Batch Size : 64
Optimisers : ADAM with Binarycrossentropy loss

# Combination 10
CROSS VALIDATION
1. Network Depth : 1 Hidden Layer of 64 units
2. Regularisations : Dropout of ratio 0.25 
3. Activations : Sigmoid for every hidden layer (singmoid for final layer)
4. Batch Size : 64
5. Optimisers : ADAM with Binarycrossentropy loss

In [604]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(64, init = "uniform", activation = 'relu',input_dim = 6))
    classifier.add(Dropout(p = 0.25))
    classifier.add(Dense(output_dim = 1, init = "uniform", activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [605]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 64, epochs = 20)

In [606]:
accuracies_64 = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [607]:
mean_64 = accuracies_64.mean()
variance_64 = accuracies_64.std()
mean_64

0.6112179696559906

In [599]:
variance_64

0.0451857535621999

In [608]:
loss_comb10, accuracy_comb10 = model.evaluate(X_test, y_test)
print("Test Loss:", loss_comb10)
print("Test Accuracy:", accuracy_comb10)

262/262 [==============================] - 0s 100us/sample - loss: 0.4787 - acc: 0.7939
Test Loss: 0.4786671927867045
Test Accuracy: 0.79389316


In [601]:
statastics.append([loss_comb10,accuracy_comb10*100])

In [602]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_test)
print('Confusion Matrix:\n')
print(cm)
print('\n')

Confusion Matrix:

[[153  16]
 [ 38  55]]




In [603]:
inc=1
print('--------------------------------------------------')
print(f'Combination  |  Test LOSS         |    Test Accuracy')
print('--------------------------------------------------')
for item in statastics:
  print(f'>    {inc}        {item[0]}     {item[1]}')
  print('--------------------------------------------------')
  inc = inc + 1

--------------------------------------------------
Combination  |  Test LOSS         |    Test Accuracy
--------------------------------------------------
>    1        0.4696569856796556     78.24427485466003
--------------------------------------------------
>    2        0.4751596837553359     80.15267252922058
--------------------------------------------------
>    3        0.4384832750749952     80.5343508720398
--------------------------------------------------
>    4        0.4661302698477534     80.15267252922058
--------------------------------------------------
>    5        0.45638758717602446     80.5343508720398
--------------------------------------------------
>    6        0.4780400067795324     77.86259651184082
--------------------------------------------------
>    7        0.4543302027323774     79.00763154029846
--------------------------------------------------
>    8        0.4786671927867045     79.38931584358215
-------------------------------------------------

We have executed 10 different combinations varying the number of units, Activation functions, Optimiser and regulariser.

The dataset is not a very big one and so stark differences may not be observed within each combination

**Below are the key observations:**

**The best performers are - Observation 3 and observation 5**
This is due to the fact that we have used RELU activation function in each of these.

**The worst performance was observed in Observation 6**
This is due to the fact that we have used Sigmoid activation function within the hidden layers.

**CONCLUSIONS:**

RELU as a function is good in the aspect that it reduces the chance of gradient vanishing Gradient 
It is computationaly more effectient as it doesnt need to do any complex computations .

SIGMOID as a function tends to vanish the Gradient and also is a complex calculation comparing RELU function
Its good for Binary output layer 
